In [1]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
from scipy import stats
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.model_selection import KFold, cross_val_score 
import statsmodels.api as sm
from sqlalchemy import create_engine
import sqlite3


In [2]:
# Verify sqlite3
print("sqlite3 version:", sqlite3.sqlite_version)

# Verify SQLAlchemy
engine = create_engine('sqlite:///test.db')
print("SQLAlchemy engine created:", engine)

sqlite3 version: 3.45.3
SQLAlchemy engine created: Engine(sqlite:///test.db)


## Define the List of CSV Files and Set Up Directories

In [3]:
# Set the data folder path
data_folder = 'data'

# List all CSV files in the data folder
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Create directories if they don't exist
os.makedirs('cleaned_data', exist_ok=True)
os.makedirs('text_data', exist_ok=True)
os.makedirs('images', exist_ok=True)

##  Extract and Clean Data

In [4]:
# Set future behavior for downcasting in replace
pd.set_option('future.no_silent_downcasting', True)

# Function to clean and save a single CSV file
def clean_and_save_csv(file_path):
    # Read the CSV file with low_memory to handle mixed data types
    df = pd.read_csv(file_path, low_memory=False)
    
    # Replace blank cells with NaN, "Needs Evaluation" with NaN, "Fail" with 0, and "Pass" with 1
    df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

    # Apply the old behavior for downcasting in replace
    result = df.replace({"Needs Evaluation": np.nan, "Fail": 0, "Pass": 1})
    result.infer_objects(copy=False)
    
    # Clean column names
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    
    # Define cleaned file path
    cleaned_file_path = os.path.join('cleaned_data', os.path.basename(file_path))
    
    # Save cleaned DataFrame
    df.to_csv(cleaned_file_path, index=False)
    print(f"Cleaned file saved to: {cleaned_file_path}")
    
    return df

# Process each CSV file and store the cleaned DataFrames
cleaned_dataframes = [clean_and_save_csv(os.path.join(data_folder, file)) for file in csv_files]

Cleaned file saved to: cleaned_data\C100.csv
Cleaned file saved to: cleaned_data\C200.csv
Cleaned file saved to: cleaned_data\C300.csv
Cleaned file saved to: cleaned_data\C400.csv
Cleaned file saved to: cleaned_data\C500.csv
Cleaned file saved to: cleaned_data\F100.csv
Cleaned file saved to: cleaned_data\H100.csv
Cleaned file saved to: cleaned_data\L100.csv
Cleaned file saved to: cleaned_data\M000.csv
Cleaned file saved to: cleaned_data\M100.csv
Cleaned file saved to: cleaned_data\M200.csv
Cleaned file saved to: cleaned_data\M300.csv
Cleaned file saved to: cleaned_data\M400.csv
Cleaned file saved to: cleaned_data\S100.csv


C:\Users\balla\AppData\Local\Temp\ipykernel_19940\4101726831.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = df.replace({"Needs Evaluation": np.nan, "Fail": 0, "Pass": 1})
C:\Users\balla\AppData\Local\Temp\ipykernel_19940\4101726831.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = df.replace({"Needs Evaluation": np.nan, "Fail": 0, "Pass": 1})


## Transform Data

In [ ]:
# Load all cleaned CSV files into a list of DataFrames with block information
data_frames = []
for file in csv_files:
    block_name = file.split('.')[0]  # Extract block name from filename (before the extension)
    df = pd.read_csv(os.path.join('cleaned_data', file))
    
    # Create the unique identifier
    df['unique_id'] = df['last_name'].str[0] + df['first_name'].str[0] + df['learnerID'].astype(str)
    df['block'] = block_name  # Add block name as a new column
    data_frames.append(df)

# Consolidate all DataFrames into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# Handle missing values and data types as necessary
combined_df.fillna(0, inplace=True)  # Example of handling missing values
combined_df['team'] = combined_df['team'].astype(int)  # Ensure 'team' column is integer

# Create helper columns if needed
combined_df['team_section'] = combined_df['team'].astype(str) + combined_df['section']

# Automatically identify numeric columns for analysis
numeric_columns = combined_df.select_dtypes(include=[np.number]).columns.tolist()

 ## Load Data into a Database

In [ ]:
# Database connection string (example using SQLite)
db_connection_string = 'sqlite:///data/transformed_data.db'
engine = create_engine(db_connection_string)

# Load data into the database
combined_df.to_sql('consolidated_table', engine, index=False, if_exists='replace')

print("Data loaded successfully into the database.")

## Conduct Analysis Using SQL

In [ ]:
# Database connection string (example using SQLite)
db_connection_string = 'sqlite:///data/transformed_data.db'
engine = create_engine(db_connection_string)

# Load data into the database
combined_df.to_sql('consolidated_table', engine, index=False, if_exists='replace')

print("Data loaded successfully into the database.")

# Generate SQL Queries Dynamically

In [ ]:
def generate_sql_query(group_by_col, numeric_cols):
    # Generate the SQL query dynamically based on the numeric columns
    select_statements = [f"AVG({col}) as avg_{col}, MEDIAN({col}) as median_{col}, STDDEV({col}) as std_{col}, " \
                         f"COUNT({col}) as count_{col}, MIN({col}) as min_{col}, MAX({col}) as max_{col}" for col in numeric_cols]
    query = f"""
    SELECT {group_by_col},
           {', '.join(select_statements)}
    FROM consolidated_table
    GROUP BY {group_by_col};
    """
    return query

# Connect to the database
conn = sqlite3.connect('data/transformed_data.db')

# Define functions for analysis
def analyze_by_team(numeric_cols):
    query = generate_sql_query('team', numeric_cols)
    return pd.read_sql(query, conn)

def analyze_by_section(numeric_cols):
    query = generate_sql_query('team_section', numeric_cols)
    return pd.read_sql(query, conn)

def analyze_by_individual(numeric_cols):
    query = generate_sql_query('unique_id, block', numeric_cols)
    return pd.read_sql(query, conn)

def correlation_analysis():
    query = "SELECT * FROM consolidated_table"
    df = pd.read_sql(query, conn)
    return df.corr()


# Conduct Analysis and Generate Visualizations Dynamically

In [ ]:
# Conduct analysis for each block
blocks = combined_df['block'].unique()  # Get unique blocks
analysis_results = {}
for block in blocks:
    block_df = combined_df[combined_df['block'] == block]
    team_performance = analyze_by_team(numeric_columns)
    section_performance = analyze_by_section(numeric_columns)
    individual_performance = analyze_by_individual(numeric_columns)
    correlation_matrix = correlation_analysis()
    
    # Store results in a dictionary for easy access later
    analysis_results[block] = {
        'team_performance': team_performance,
        'section_performance': section_performance,
        'individual_performance': individual_performance,
        'correlation_matrix': correlation_matrix
    }
    
    # Write the analysis results to a text file
    with open(f'text_data/{block}_analysis.txt', 'w') as file:
        file.write(f'Analysis for {block}\n')
        file.write('Team Performance:\n')
        file.write(team_performance.to_string())
        file.write('\n\nSection Performance:\n')
        file.write(section_performance.to_string())
        file.write('\n\nIndividual Performance:\n')
        file.write(individual_performance.to_string())
        file.write('\n\nCorrelation Matrix:\n')
        file.write(correlation_matrix.to_string())
    
    # Create and save visualizations for team performance
    for col in numeric_columns:
        plt.figure(figsize=(10, 6))
        sns.barplot(x=team_performance['team'], y=team_performance[f'avg_{col}'] )
        plt.title(f'Team Performance in {block.capitalize()} ({col})')
        plt.savefig(f'images/{block}_team_performance_{col}.png')
        plt.close()

    # Create and save visualizations for section performance
    for col in numeric_columns:
        plt.figure(figsize=(10, 6))
        sns.barplot(x=section_performance['team_section'], y=section_performance[f'avg_{col}'])
        plt.title(f'Section Performance in {block.capitalize()} ({col})')
        plt.savefig(f'images/{block}_section_performance_{col}.png')
        plt.close()
